In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pathlib import Path
from glob import glob

path_start = Path("/content/drive/MyDrive/Nishika/Mansion_2023_spring")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r "/content/drive/MyDrive/Nishika/Mansion_2023_spring/input/train.zip" .
!unzip "/content/drive/MyDrive/Nishika/Mansion_2023_spring/input/train.zip"

Archive:  /content/drive/MyDrive/Nishika/Mansion_2023_spring/input/train.zip
replace train/41.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
TRAIN_PATH = Path("/content/train")
df_MERGE = pd.DataFrame()

for i in glob(str(TRAIN_PATH/"*")):
  df_MERGE = pd.concat([df_MERGE, pd.read_csv(i)])

df_MERGE.head()

<ipython-input-3-e9815f137182>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_MERGE = pd.concat([df_MERGE, pd.read_csv(i)])
<ipython-input-3-e9815f137182>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_MERGE = pd.concat([df_MERGE, pd.read_csv(i)])
<ipython-input-3-e9815f137182>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_MERGE = pd.concat([df_MERGE, pd.read_csv(i)])
<ipython-input-3-e9815f137182>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_MERGE = pd.concat([df_MERGE, pd.read_csv(i)])


,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,8067540,中古マンション等,NaN,8217,茨城県,取手市,野々井,稲戸井,13,３ＬＤＫ,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2007年第3四半期,未改装,NaN,6.977724
1,8027265,中古マンション等,NaN,8219,茨城県,牛久市,ひたち野東,ひたち野うしく,8,３ＬＤＫ,...,NaN,NaN,NaN,第２種中高層住居専用地域,60.0,200.0,2018年第3四半期,未改装,NaN,7.255273
2,8061526,中古マンション等,NaN,8220,茨城県,つくば市,苅間,研究学園,4,３ＬＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2011年第4四半期,未改装,NaN,7.556303
3,8086147,中古マンション等,NaN,8220,茨城県,つくば市,並木,つくば,1H?1H30,４ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,200.0,2013年第1四半期,未改装,NaN,7.380211
4,8049498,中古マンション等,NaN,8224,茨城県,守谷市,ひがし野,守谷,8,４ＬＤＫ,...,NaN,NaN,NaN,近隣商業地域,80.0,200.0,2018年第2四半期,未改装,NaN,7.518514


In [ ]:
df_MERGE.shape

(765412, 28)

In [ ]:
test = pd.read_csv(path_start/"input/test.csv")
df_ConCat = pd.concat([df_MERGE, test])

In [ ]:
test.shape

(19301, 27)

In [ ]:
df_ConCat["面積（㎡）"] = df_ConCat["面積（㎡）"].replace("2000㎡以上", 2000)
df_ConCat["面積（㎡）"] = df_ConCat["面積（㎡）"].astype(int)

In [ ]:
df_ConCat["最寄駅：距離（分）"] = df_ConCat["最寄駅：距離（分）"].replace("30分?60分", 45)
df_ConCat["最寄駅：距離（分）"] = df_ConCat["最寄駅：距離（分）"].replace("1H?1H30", 75)
df_ConCat["最寄駅：距離（分）"] = df_ConCat["最寄駅：距離（分）"].replace("1H30?2H", 105)
df_ConCat["最寄駅：距離（分）"] = df_ConCat["最寄駅：距離（分）"].replace("2H?", 120)
df_ConCat["最寄駅：距離（分）"] = df_ConCat["最寄駅：距離（分）"].fillna(-1)
df_ConCat["最寄駅：距離（分）"] = df_ConCat["最寄駅：距離（分）"].astype(int)

In [17]:
df_ConCat["最寄駅：距離（分）"].unique()

array([ 13,   8,   4,  75,   5,  45,  -1,  10,  25,   1,  28, 120,  11,
         9,  24,  12,  16,   6,   3,  20,  15,  23,  19,  26,   7,  14,
         2,  21,  18,   0,  29,  22, 105,  17,  27])

In [ ]:
def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

Seireki = lambda x: convert_wareki_to_seireki(x)

In [ ]:
df_ConCat["建築年"] = df_ConCat["建築年"].apply(Seireki)

In [ ]:
test = df_ConCat[df_ConCat["取引価格（総額）_log"].isna()==True]
df_train = df_ConCat[df_ConCat["取引価格（総額）_log"].isna()!=True]

In [ ]:
df_train.to_csv(path_start/"input/df_train.csv")
test.to_csv(path_start/"input/test.csv")